In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
import nltk
import tiktoken
nltk.download('stopwords')
nltk.download('punkt')
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mojee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mojee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
df = pd.read_csv("data/v2.csv", sep=";", )
df.tail()

,event,comment
65,Lionel Messi winning the World Cup,"""He was beautiful ,he was the point of differ..."
66,Christiano Ronaldo's goal,"""Cristiano Ronaldo scores again! What a legen..."
67,Christiano Ronaldo's goal,"""He plays off his left foot, he almost doesn'..."
68,Christiano Ronaldo's goal,"""The man is a storybook all by himself. There..."
69,Christiano Ronaldo's goal for Manchester United,"""Ole Gunnar Solskjaer gives thanks again for ..."


In [4]:
train_test_ratio = 0.2

train_set, test_set = train_test_split(df, test_size=train_test_ratio)

In [5]:
test_set

,event,comment
54,Tottenham win against Manchester City,"""Pep's Manchester City, beaten in the Premier..."
50,Manchester city vs Tottenham,"""The hurricane that hit Manchester for the be..."
13,Spain vs Russia,"""Spain on the plane, or Spain in the Russian ..."
55,Tottenham win against Manchester City,"""Hotspur's game plan was spot without blemish..."
49,Paul Walker comparison with Son-Heung Min,"""Walker is adhesive Son Heung-Min is elusive."""
10,Scott McTominay's score for Manchester United ...,"""A red-Letter day, Scott McTominay, Ederson s..."
23,Sergio Aguero's goal for Manchester City vs Qu...,"""Drama of the ultimate type! Tears of distres..."
9,Peter Drury's commentary on Roma,"""Roma have risen from their ruins. Manolas, a..."
45,Manchester City vs Leicester City. Vincent Kom...,"""Kompany... had a look, had a hit... ohhh! Ca..."
12,"Germany win against Brazil, 2014 FIFA World Cup","From Chancellor to the cheap seats, the German..."


In [7]:
X_train = train_set['event']
y_train = train_set[' comment']
X_test = test_set['event']
y_test = test_set[' comment']

In [17]:
X_train

22              Manchester City win against Aston Villa
42    South Africa vs Mexico. Siphiwe Tshabalala's H...
36                        Cameroon score against Brazil
29                            Kylian Mbappe's hat trick
34                   Senegal vs Ecuador, FIFA world cup
28                                 Kylian Mbappe's goal
62                   Lionel Messi winning the World Cup
37                                   Morocco vs Belgium
38                                  Argentina vs Mexico
35                                   Cameroon vs Brazil
8                         Peter Drury on Etihad stadium
58                                  Lionel Messi's goal
59                                  Lionel Messi's goal
40        England vs Colombia. Eric Dier's Penalty goal
15                                     Germany vs Korea
18    Aaron Ramsey's Departure from Arsenal to Juventus
7         Peter Drury's commentary on Cristiano Ronaldo
3     England's defeat to Croatia in the 2018 Wo

In [9]:
y_train

22     "From the brink of despair, elation. How dare...
42     "It's Tshabalala... Goal Bafana Bafana! Goal ...
36     "Ooooooh! Cameroon! From Douala on the Gulf o...
29     "One feels for Mbappe, who scored a World Cup...
34     "Captain fantastic Kalidou Koulibally, the pe...
28                  "Mbappe…Into the galaxy of greats!"
62     "The greatest player of his age finally has t...
37     "Drink it in Casablanca, relish it Rabat, thi...
38     "Perhaps Argentina's motto is - 'it is not ho...
35     "The Indomitable Lions have been tamed by the...
8      "Etihad is bubbling up like it only does on '...
58     "Lionel Messi dribbles past defenders effortl...
59     "Another incredible goal by Messi! He makes i...
40     "History has been made for this new team, new...
15     "It seems like the end of the world, and it c...
18     "One day, next autumn when Aaron Ramsey wakes...
7      "The man is a storybook all by himself. There...
3      "The Three Lions were toothless voiceless

In [10]:
X_test

54                Tottenham win against Manchester City
50                         Manchester city vs Tottenham
13                                      Spain vs Russia
55                Tottenham win against Manchester City
49            Paul Walker comparison with Son-Heung Min
10    Scott McTominay's score for Manchester United ...
23    Sergio Aguero's goal for Manchester City vs Qu...
9                      Peter Drury's commentary on Roma
45    Manchester City vs Leicester City. Vincent Kom...
12      Germany win against Brazil, 2014 FIFA World Cup
68                            Christiano Ronaldo's goal
0     South Africa taking the lead against Mexico in...
39                               Netherlands vs Senegal
25    Bukayo Saka's Penalty kick for Arsenal against...
Name: event, dtype: object

In [11]:
y_test

54     "Pep's Manchester City, beaten in the Premier...
50     "The hurricane that hit Manchester for the be...
13     "Spain on the plane, or Spain in the Russian ...
55     "Hotspur's game plan was spot without blemish...
49       "Walker is adhesive Son Heung-Min is elusive."
10     "A red-Letter day, Scott McTominay, Ederson s...
23     "Drama of the ultimate type! Tears of distres...
9      "Roma have risen from their ruins. Manolas, a...
45     "Kompany... had a look, had a hit... ohhh! Ca...
12    From Chancellor to the cheap seats, the German...
68     "The man is a storybook all by himself. There...
0     Goal Bafana Bafana! Goal for South Africa! Goa...
39     "De Jong… Goalkeeper's come and Holland lead…...
25     "Now this is one brave boy. Bukayo Saka. Beca...
Name:  comment, dtype: object

In [12]:
categories = [
    'comp.graphics',
    'comp.os.ms-windows.misc',
    'rec.sport.baseball',
    'rec.sport.hockey',
    'alt.atheism',
    'soc.religion.christian',
]

dataset = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, remove=('headers', 'footers', 'quotes'))
corpus = [item for item in dataset['data'][:10]]
corpus

["\nThey tried their best not to show it, believe me. I'm surprised they\ncouldn't find a sprint car race (mini cars through pigpens, indeed!)\non short notice.\n\nGeorge",
 '\nStankiewicz?  I doubt it.\n\nKoufax was one of two Jewish HOFs: the other is Hank Greenberg.\n\nOther good players: Buddy Myer, Johnny Kling, Norm and Larry Sherry,\nKen Holtzman, Saul Rogovin, Ed Reulbach.  \n\nThere have been over 150 Jewish major leaguers.  A few years ago there\nwas an article about someone who keeps track of this in Spy magazine;\nthe article was entitled "Jews on First," of course.\n\nThere have also been at least two books on the subject.',
 '\n[deletia- and so on]\n\nI seem to have been rather unclear.\n\nWhat I was asking is this:\n\nPlease show me that the most effective substance-absure recovery\nprograms involve meetinsg peoples\' spiritual needs, rather than\nmerely attempting to fill peoples\' spiritual needs as percieved\nby the people, A.A, S.R.C. regulars, or snoopy. This will p

In [13]:
encoding = tiktoken.get_encoding("cl100k_base")

In [15]:
text = "Hello world. Lorem ipsum dolor sit"
toks = encoding.encode(text)
tk = encoding.decode(toks)
tk

'Hello world. Lorem ipsum dolor sit'

## Tokenization
Using `tiktoken` library

In [24]:
X_train_tokens = []
for i in X_train:
    enc = encoding.encode(i)
    X_train_tokens.append(enc)
X_test_tokens = []
for i in X_test:
    enc = encoding.encode(i)
    X_test_tokens.append(enc)
y_train_tokens = []
y_test_tokens = []
for i in y_train:
    enc = encoding.encode(i)
    y_train_tokens.append(enc)
for i in y_test:
    enc = encoding.encode(i)
    y_test_tokens.append(enc)

## Tokens to tensors

In [29]:
X_train_tensors = [torch.tensor(data) for data in X_train_tokens]
y_train_tensors = [torch.tensor(data) for data in y_train_tokens]
X_test_tensors = [torch.tensor(data) for data in X_test_tokens]
y_test_tensors = [torch.tensor(data) for data in y_test_tokens]

In [43]:
X_train_tensors_padded = pad_sequence(X_train_tensors, batch_first=True)
X_train_tensors_padded.shape


torch.Size([56, 16])

In [44]:
y_train_tensors_padded = pad_sequence(y_train_tensors, batch_first=True)
y_train_tensors_padded.shape

torch.Size([56, 92])

In [40]:
X_test_tensors_padded = pad_sequence(X_test_tensors, batch_first=True)
X_test_tensors_padded.shape

torch.Size([14, 14])

In [39]:
y_test_tensors_padded = pad_sequence(y_test_tensors, batch_first=True)
y_test_tensors_padded.shape

torch.Size([14, 80])

In [42]:
max_length = max(X_train_tensors_padded.shape, X_test_tensors_padded.shape)
max_length

torch.Size([56, 16])

In [ ]:
class Network(nn.Module):
    def __init__(self, x_):
        super(Network, self).__init__()
    
    def forward(self, x):
        
        return x